In [2]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
feim = pd.read_csv('../output/valid_single_feature_tochi.csv', header=None)
# feim = pd.read_csv('../output/valid_single_feature.csv', header=None)
feim.columns = ['datetime', 'DT-M', 'feature', 'score']

# feim = feim[feim['datetime']<='20190913_22380']
print(feim['datetime'].max())

feim.set_index('feature', inplace=True)
cnt = feim.groupby('feature')['score'].count()
idx = cnt[cnt==3].index
feim = feim.loc[idx]
may = feim[feim['DT-M']=='2018-5']
may.sort_values(by='score', ascending=False, inplace=True)
feim.loc[may.index]

20190914_05311


,datetime,DT-M,score
feature,,,
603__card2-card5_V243__ProductCD-W_std_train,20190914_03420,2018-5,0.936761
603__card2-card5_V243__ProductCD-W_std_train,20190914_03420,2018-4,0.946325
603__card2-card5_V243__ProductCD-W_std_train,20190914_03420,2018-3,0.935212
603__card4-addr2_C13__ProductCD-C_std_train,20190914_04480,2018-5,0.936537
603__card4-addr2_C13__ProductCD-C_std_train,20190914_04480,2018-4,0.947337
603__card4-addr2_C13__ProductCD-C_std_train,20190914_04480,2018-3,0.936333
509__C11-V78__ratio__ProductCD-H_train,20190914_03393,2018-5,0.936127
509__C11-V78__ratio__ProductCD-H_train,20190914_03393,2018-4,0.948077
509__C11-V78__ratio__ProductCD-H_train,20190914_03393,2018-3,0.935648
